RAG - REACT - TOOL

In [27]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True) 

True

In [84]:
def get_embed_model():
    from jproperties import Properties 
    configs = Properties() 
  
    with open('sample.properties', 'rb') as read_prop: 
        configs.load(read_prop) 
      
    prop_view = configs.items() 
    for item in prop_view:
        if item[0] == 'embed_model':
            print(item[0], ':', item[1].data)
            embed_model = item[1].data
    return embed_model

def get_llm_model():
    from jproperties import Properties 
    configs = Properties() 
  
    with open('sample.properties', 'rb') as read_prop: 
        configs.load(read_prop) 
    prop_view = configs.items() 
    for item in prop_view: 
        if item[0] == 'llm_model':
            print(item[0], ':', item[1].data)
            llm_model = item[1].data
    return llm_model

In [74]:
def create_embeddings_chroma(chunks, persist_directory='./chroma_db'):
    from langchain.vectorstores import Chroma
    from langchain_openai import OpenAIEmbeddings

    # Instantiate an embedding model from OpenAI (smaller version for efficiency)
    #embeddings = OpenAIEmbeddings(model='text-embedding-3-small', dimensions=1536)  
    embed_model = get_embed_model()
    embeddings = OpenAIEmbeddings(model=embed_model, dimensions=1536)  

    # Create a Chroma vector store using the provided text chunks and embedding model, 
    # configuring it to save data to the specified directory 
    vector_store = Chroma.from_documents(chunks, embeddings, persist_directory=persist_directory) 

    return vector_store  # Return the created vector store

In [75]:
def load_embeddings_chroma(persist_directory='./chroma_db'):
    from langchain.vectorstores import Chroma
    from langchain_openai import OpenAIEmbeddings

    # Instantiate the same embedding model used during creation
    #embeddings = OpenAIEmbeddings(model='text-embedding-3-small', dimensions=1536) 
    embed_model = get_embed_model()
    embeddings = OpenAIEmbeddings(model=embed_model, dimensions=1536) 

    # Load a Chroma vector store from the specified directory, using the provided embedding function
    vector_store = Chroma(persist_directory=persist_directory, embedding_function=embeddings) 

    return vector_store  # Return the loaded vector store

In [76]:
# Loading the pdf document into LangChain 
#Load the documents
from langchain_community.document_loaders import DirectoryLoader
DATA_PATH = "mydata"
loader = DirectoryLoader(DATA_PATH, glob="*.pdf")
data = loader.load()

In [77]:
def chunk_data(data, chunk_size, chunk_overlap): #make this 512 anc see u shall not get the result
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = text_splitter.split_documents(data)
    return chunks

In [78]:
from langchain.vectorstores import Chroma
# Splitting the document into chunks
chunks = chunk_data(data, chunk_size=500, chunk_overlap=50) 

In [79]:
# Creating a Chroma vector store using the provided text chunks and embedding model (default is text-embedding-3-small)
vector_store = create_embeddings_chroma(chunks)

embed_model : text-embedding-3-small


In [85]:
def ask_and_get_answer(vector_store, q, k=5):
    from langchain.chains import RetrievalQA
    from langchain_openai import ChatOpenAI

    llm_model = get_llm_model()
    llm = ChatOpenAI(model=llm_model, temperature=0)
    #llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)

    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': k})

    chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
    
    answer = chain.invoke(q)
    return answer

In [86]:
# Asking questions
#question = 'Does WMi support json interfaces?'
#question = 'Who is the current prime minister of the UK?'
#question = 'what is the answer to 3 ** 3.9'
#question = 'Tell me about Napoleon Bonaparte early life'
#question = 'how to enable "Putaway To Empty Non-Replenishable Dynamic Active Locations" in Manhattan Active Supply Chain'
#question = 'how to enable "Putaway To Empty Non-Replenishable Dynamic Active Locations" in Manhattan Active® Warehouse Management?'
#question = 'Is the feature "Creation of Multiple Process Needs for an Item" in Manhattan Active Supply Chain, enabled by default?'
#question = 'Is the feature "Creation of Multiple Process Needs for an Item" in Manhattan Active® Warehouse Management, enabled by default?'
#question = 'What is the maximum weight and volume supported for transportaion order size in Manhattan Active TM?'
#question = 'what is the default maximum weight and volume supported?'
question = 'what is the default maximum weight supported in lbs?'
#question = 'Explain transportation order alerts and missing fields'

vector_store = load_embeddings_chroma()
answer = ask_and_get_answer(vector_store, question)
#print(answer)
print(answer['result'])

embed_model : text-embedding-3-small
llm_model : gpt-3.5-turbo
The default maximum weight supported in lbs is 40000 lbs.


In [87]:
# We can't ask follow-up questions. There is no memory (chat history) available.
question = 'Multiply that number by 2.'
answer = ask_and_get_answer(vector_store, question)
print(answer['result'])

llm_model : gpt-3.5-turbo
I don't have a specific number to multiply by 2 based on the context provided.


In [88]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain  # Import class for building conversational AI chains 
from langchain.memory import ConversationBufferMemory  # Import memory for storing conversation history

# Instantiate a ChatGPT LLM (temperature controls randomness)
llm_model = get_llm_model()
#llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0) 
llm = ChatOpenAI(model_name=llm_model, temperature=0)  

# Configure vector store to act as a retriever (finding similar items, returning top 5)
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 5})  


# Create a memory buffer to track the conversation
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

crc = ConversationalRetrievalChain.from_llm(
    llm=llm,  # Link the ChatGPT LLM
    retriever=retriever,  # Link the vector store based retriever
    memory=memory,  # Link the conversation memory
    chain_type='stuff',  # Specify the chain type
    verbose=False  # Set to True to enable verbose logging for debugging
)

llm_model : gpt-3.5-turbo


In [89]:
# create a function to ask questions
def ask_question(q, chain):
    result = chain.invoke({'question': q})
    return result

In [90]:
# Asking questions
#question = 'Does WMi support json interfaces?'
#question = 'Who is the current prime minister of the UK?'
#question = 'what is the answer to 3 ** 3.9'
#question = 'Tell me about Napoleon Bonaparte early life'
#question = 'how to enable "Putaway To Empty Non-Replenishable Dynamic Active Locations" in Manhattan Active Supply Chain'
#question = 'how to enable "Putaway To Empty Non-Replenishable Dynamic Active Locations" in Manhattan Active® Warehouse Management?'
#question = 'Is the feature "Creation of Multiple Process Needs for an Item" in Manhattan Active Supply Chain, enabled by default?'
#question = 'Is the feature "Creation of Multiple Process Needs for an Item" in Manhattan Active® Warehouse Management, enabled by default?'
#question = 'What is the maximum weight and volume supported for transportaion order size in Manhattan Active TM?'
#question = 'what is the default maximum weight and volume supported?'
question = 'what is the default maximum weight supported in lbs?'
#question = 'Explain transportation order alerts and missing fields'

vector_store = load_embeddings_chroma()

result = ask_question(question, crc)
print(result['answer'])

embed_model : text-embedding-3-small
The default maximum weight supported in lbs is 40000 lbs.


In [91]:
question = 'Multiply that number by 10.'
result = ask_question(question, crc)
print(result['answer'])

The result of multiplying the default maximum weight of 40000 lbs by 10 is 400,000 lbs.


In [92]:
question = 'Devide the result by 40.'
result = ask_question(question, crc)
print(result['answer'])

I don't have the specific calculation for that, as the information provided is related to payload data and not mathematical calculations.


In [93]:
for item in result['chat_history']:
    print(item)

content='what is the default maximum weight supported in lbs?'
content='The default maximum weight supported in lbs is 40000 lbs.'
content='Multiply that number by 10.'
content='The result of multiplying the default maximum weight of 40000 lbs by 10 is 400,000 lbs.'
content='Devide the result by 40.'
content="I don't have the specific calculation for that, as the information provided is related to payload data and not mathematical calculations."
